In [123]:
import re
import os
import nltk
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import tensorflow_hub as hub
import tensorflow as tf

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jsbae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [110]:
text_file = open('C:/Users/jsbae/Downloads/Qualcomm text data/QCOM_0000804328_10Q_20171224_AllItems_excerpt.txt', encoding='utf-8').read()
condensed_text = text_file[3200:]
actual_condensed = text_file[3200:7000]

lemmatizer = WordNetLemmatizer()


In [108]:
# regexed to remove decimal points to split by sentences later
new_condensed = re.sub('(?<!\d)\.(?=\d)', '', condensed_text)
new_condensed = new_condensed.replace(',', '')
new_condensed = new_condensed.replace('(', '')
new_condensed = new_condensed.replace(')', '')

# split into lines by periods
text_lines = new_condensed.split('.')
print('Number of text lines:', len(text_lines))

# eliminate short sentences (which are probably titles anyway)
long_sents = [sentence for sentence in text_lines if len(sentence) > 5]
print('Number of long_sents:', len(long_sents))

Number of text lines: 1339
Number of long_sents: 1275


In [115]:
# tokenize the sentences and lemmatize them
tokenized_sents = []
for line in long_sents:
    tokenized = re.findall('\w+(?:-\w+)*|\$[\d.]+|\S+', line)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokenized]
    tokenized_sents.append(lemmatized)

print('Length of tokenized sents:', len(tokenized_sents))

# removes (Unaudited) and [DATA_TABLE_REMOVED] in the text
for sentence in tokenized_sents:
    while '(Unaudited)' or '[DATA_TABLE_REMOVED]' in sentence:
        try:
            sentence.remove('(Unaudited)')
            sentence.remove('[DATA_TABLE_REMOVED]')
        except ValueError:
            break
            
print('total number of words:', sum([len(listElem) for listElem in tokenized_sents]))         

# remove stopwords
for sentence in tokenized_sents:
    for word in sentence:
        while word in stopwords.words('english'):
            try:
                sentence.remove(word)
            except ValueError:
                break

print('total number of words:', sum([len(listElem) for listElem in tokenized_sents])) 
print('Total tokenized sents:', len(tokenized_sents))

Length of tokenized sents: 1275
total number of words: 37084
total number of words: 25306
Total tokenized sents: 1275


In [121]:
# developing word embedding for elmo use
sents = []
for sentence in tokenized_sents:
    string = ' '.join(sentence)
    sents.append(string)
    

In [124]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
embeddings = elmo(sents, signature = "default", as_dict=True)['elmo']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [125]:
embeddings.shape

TensorShape([Dimension(1275), Dimension(345), Dimension(1024)])

In [126]:
def elmo_vectors(x):
  # embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))


In [127]:
elmo_train = elmo_vectors(embeddings)

ResourceExhaustedError: OOM when allocating tensor with shape[1275,345,44,1024] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node module_apply_default/bilm/CNN/Conv2D_6 (defined at C:\Users\jsbae\Anaconda3\lib\site-packages\tensorflow_hub\native_module.py:610) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'module_apply_default/bilm/CNN/Conv2D_6':
  File "C:\Users\jsbae\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jsbae\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jsbae\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\jsbae\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Use

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)


In [ ]:
print(missing_sents)
